In [ ]:
pip install transformers


In [ ]:
from transformers import pipeline 
print(pipeline('sentiment-analysis')('we love you'))

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import transformers as ppb
import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

data = pd.read_excel('/content/gdrive/MyDrive/Praxisprojekt/Trainingdata.xlsx', sheet_name = 'sentences')

#id 2257 ist doof
#delete later
data = data.head(2257)

data.drop(['SUBJindl', 'SUBJsrce', 'SUBJrhet', 'SUBJster', 'SUBJspee', 'SUBJinspe', 'SUBJprop', 'SUBJpolit'], axis=1, inplace=True)

model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

#Sätze zerstückeln lassen
tokenized = data['Sentence'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

#Padding hinzufügen 
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

#Maske erstellen, um das Padding bei der Verarbeitung zu filtern
mask = np.where(padded != 0, 1, 0)
mask.shape

#mache padded Array und Maske zu einem Tensor
#Tensor = irgendeine mehrdimensionale Matrix-Repräsentation
input = torch.tensor(padded)  
mask = torch.tensor(mask)

#gib unser Zeug an BERT
#no_grad = Angabe zur Simplifikation des Rechenvorgangs
with torch.no_grad():
    output = model(input, attention_mask=mask)

#nur die erste Spalte auslesen = von BERT geschriebene Kennwerte
features = output[0][:,0,:].numpy()

In [ ]:
from sklearn.naive_bayes import GaussianNB

#labels holen
labels = data[['SUBJlang', 'SUBJopin']]

label1 = labels['SUBJlang']
label2 = labels['SUBJopin']

#und jetzt alles durch die Toolbox von Scikit jagen
train_features, test_features, train_labels, test_labels = train_test_split(features, label1)

lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

#Score
print("Score SUBJlang")
print(lr_clf.score(test_features, test_labels))

#und jetzt NOCHMAL alles durch die Toolbox von Scikit jagen
train_features, test_features, train_labels, test_labels = train_test_split(features, label2)

lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

#Score 2
print("Score SUBJopin")
print(lr_clf.score(test_features, test_labels))

#und JETZT Naive Bayes

train_features, test_features, train_labels, test_labels = train_test_split(features, label1)

nb_clf = GaussianNB()
nb_clf.fit(train_features, train_labels)

print("Score: Naive Bayes")
print("SUBJlang")
print(nb_clf.score(test_features, test_labels))

train_features, test_features, train_labels, test_labels = train_test_split(features, label2)

nb_clf = GaussianNB()
nb_clf.fit(train_features, train_labels)

print("SUBJopin")
print(nb_clf.score(test_features, test_labels))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Score SUBJlang
0.6070796460176991
Score SUBJopin
0.7256637168141593
Score: Naive Bayes
SUBJlang
0.46017699115044247
SUBJopin
0.5451327433628319


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
import pandas as pd
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
df = df.head(3)
print(df[0])

0    a stirring , funny and finally transporting re...
1    apparently reassembled from the cutting room f...
2    they presume their audience wo n't sit still f...
Name: 0, dtype: object
